In [1]:
stock = 'T'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    30.644123
2018-07-23    30.545588
2018-07-24    31.215620
2018-07-25    29.806581
2018-07-26    29.905117
2018-07-27    30.624414
2018-07-30    31.530930
2018-07-31    31.501369
2018-08-01    31.432396
2018-08-02    31.383129
2018-08-03    31.796972
2018-08-06    31.639318
2018-08-07    31.925068
2018-08-08    31.885653
2018-08-09    32.013748
2018-08-10    31.787117
2018-08-13    31.737850
2018-08-14    31.767414
2018-08-15    31.994041
2018-08-16    32.604954
2018-08-17    32.545830
2018-08-20    32.624660
2018-08-21    32.910408
2018-08-22    32.200962
2018-08-23    32.013748
2018-08-24    32.161549
2018-08-27    32.348766
2018-08-28    32.191105
2018-08-29    31.757557
2018-08-30    31.491514
                ...    
2018-09-17    33.127182
2018-09-18    33.225719
2018-09-19    32.880848
2018-09-20    32.949821
2018-09-21    33.284836
2018-09-24    33.412933
2018-09-25    33.265129
2018-09-26    32.861137
2018-09-27    32.930111
2018-09-28    33.087772
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


32.625492439195455
[33.94036328 33.15663538 33.24432746 31.06921065 31.71692543]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")